## 자연어 전처리-영어

모델생성

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer #전처리 : 토큰화
from tensorflow.keras.preprocessing.sequence import pad_sequences # 시퀸스 패딩이지만 정확한 용도 ????????????????
from sklearn.preprocessing import LabelEncoder # ????
from sklearn.model_selection import train_test_split # 테스트모델과 트레이닝 모델 생성
from tensorflow.keras.utils import to_categorical #y.shape 변형하는데 사용했지만 정확한 용도는 ??????????????? 
import numpy as np
from nltk.corpus import stopwords # 불용어처리
from bs4 import BeautifulSoup #html 파서 ??????
import re #???????????

전처리단계

In [ ]:
def preprocessing(X_text, remove_stopwords=False):
    X_text = BeautifulSoup(X_text, 'lxml').get_text()#문서에서 단어를 가져옴
    X_text = re.sub("[^a-zA-Z]", " ", X_text) # 영어 소문자에서 대문자 제외하고 전부 공백처리???????
    words = X_text.lower().split()#x데이터 소문자화 공백구분
    if remove_stopwords:
        stops = set(stopwords.words('english'))#영어 불용어 단어 리스트
        #stops.add(불용어 문자열)
        words = [w for w in words if not w in stops]#불용어가 없으면 리스트 등록
        clean_text = ' '.join(words)#등록된 리스트 목록을 공백기준으로 병합
    else:
        clean_text = ' '.join(words)
        
    return clean_text

모델불러오기

In [3]:
import pandas as pd
data = pd.read_csv('spam.csv',encoding='latin1')[['v1','v2']]#데이터 v1과 v2를 불러옴
data=data.rename(columns = {'v1':'y', 'v2' : 'X'}) # 데이터 이름변경

불용어처리하기ㅡ

In [4]:
data['clean_X']=data['X'].apply(lambda x : preprocessing(X_text=x, remove_stopwords=True))
#불러온 data의 x값을위의 전처리험수를 사용해 불용어 소문자 영어단어만 블러와준다.
#data['X'].apply이부분은 무슨 용도인지 ???????????????

C:\Users\gurwl\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [5]:
data['y_name']=data['y']
data['encoder_y']=LabelEncoder().fit_transform(data['y'])# LabelEncoder()?????????
data['categorical_y']=list(to_categorical(data['encoder_y']))#y.shape 변형하는데 사용했지만 정확한 용도는????????????

In [6]:
data.isnull().values.any() #data파일에 공백 값이 존재하는지 확인

False

In [7]:
data['clean_X'].nunique(), data['y'].nunique() # 값중 서로다른값이 몇개인지 확인

(5055, 2)

In [8]:
data['clean_X'] = data['clean_X'].str.replace("[^a-zA-Z0-9 ]","")#영대문자,숫자,공백 제외한 문자는 공백처리
data['clean_X'] = data['clean_X'].str.replace('^ +', "")#문장 첫단 공백제거
data['clean_X'].replace('', np.nan, inplace=True)#공백을 nan으로 변경
data = data.dropna(how = 'any')#nan값의 줄을 빼낸다

C:\Users\gurwl\AppData\Local\Temp\ipykernel_13220\3829374632.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_X'] = data['clean_X'].str.replace("[^a-zA-Z0-9 ]","")#알파벳,숫자,공백만
C:\Users\gurwl\AppData\Local\Temp\ipykernel_13220\3829374632.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_X'] = data['clean_X'].str.replace('^ +', "") # white space 데이터를 empty value로 변경


In [9]:
Y=data['encoder_y']#y데이터지정
#Y=to_categorical(data['encoder_y'])

In [10]:
X=data['clean_X']#x데이터지정

train_test 나눌때 x->x_data로 나눈것들 다시 train,val로 나눠서 쓰는구나는 알겠는데 나누는 규칙이나 방법은 잘모르겠습니다

In [11]:
x_data,test_x,y_data,test_y = train_test_split(X,Y,test_size=0.3,random_state=0)#x,y데이터와 테스트데이터 비율 7:3,랜덤값 고정

In [12]:
train_x,val_x,train_y,val_y = train_test_split(x_data,y_data,test_size=0.2,random_state=0)#트레이닝데이터와 검증데이터 비율 8:2 

In [13]:
token=Tokenizer()#토큰단어클래스 불러옴
token.fit_on_texts(train_x)#train_x 단어들을 학습

In [14]:
token_train_x=token.texts_to_sequences(train_x)## 시퀸스화라고하는데 정확한 용도??????????? 숫자화??
token_test_x=token.texts_to_sequences(test_x)## 시퀸스화라고하는데 정확한 용도???????????
token_val_x=token.texts_to_sequences(val_x)## 시퀸스화라고하는데 정확한 용도???????????

In [15]:
train_inputs = pad_sequences(token_train_x)#패딩작업 큰값에 맞춰 나머지 빈값들 0으로 채워줌
test_inputs = pad_sequences(token_test_x)
val_inputs = pad_sequences(token_val_x)